# Introduction
* Raw CSV loaded and lightly processed. Output: two column csv columns, property address, flat tag
* Data labelled in programmatic. Output: json file of entities.
* Data programmatic output json cleaned ordered and overlaps removed. Output: json file
* **Clean json converted to dataframe and multi-addresses expanded.** Output: CSV
* Count and locate addresses
* Create address matcher and match businesses
* Classify address types

This notebook is used so that I can create the code necessary to expand the addresses so that a single property/dwelling is a single line. This means the data will follow the tidy data principles of one observation per line where an observation is what is commonly thought of as a property.

In [5]:
import json
import pandas as pd


In [3]:

with open('/tf/empty_homes_data/full_dataset_no_overlaps.json', "r") as read_file:
    all_entities = json.load(read_file)


In [6]:
all_entities = pd.json_normalize(all_entities, record_path = "labels",
                       meta = ['datapoint_id', 'text'])

In [7]:
all_entities[all_entities['datapoint_id']==94087]

,start,end,label,label_text,datapoint_id,text
357948,44,45,street_number,3,94087,"storage 17, discovery dock apartments east, 3 ..."
357949,46,63,street_name,south quay square,94087,"storage 17, discovery dock apartments east, 3 ..."
357950,65,71,city,london,94087,"storage 17, discovery dock apartments east, 3 ..."
357951,73,80,postcode,e14 9ru,94087,"storage 17, discovery dock apartments east, 3 ..."


In [26]:
all_entities[all_entities['text'].str.contains(r"58 to 66 \(even numbers\)")]

,start,end,label,label_text,datapoint_id,text,index
214985,0,8,street_number,58 to 66,56091,"58 to 66 (even numbers) great portland street,...",214985
214986,10,22,postcode,even numbers,56091,"58 to 66 (even numbers) great portland street,...",214986
214987,30,45,street_name,portland street,56091,"58 to 66 (even numbers) great portland street,...",214987
214988,47,55,street_number,52 to 66,56091,"58 to 66 (even numbers) great portland street,...",214988
214989,57,61,number_filter,even,56091,"58 to 66 (even numbers) great portland street,...",214989
214990,71,86,street_name,mortimer street,56091,"58 to 66 (even numbers) great portland street,...",214990
214991,91,98,street_number,1 to 17,56091,"58 to 66 (even numbers) great portland street,...",214991
214992,100,103,number_filter,odd,56091,"58 to 66 (even numbers) great portland street,...",214992
214993,120,137,street_name,titchfield street,56091,"58 to 66 (even numbers) great portland street,...",214993
214994,139,145,city,london,56091,"58 to 66 (even numbers) great portland street,...",214994


In [23]:
all_entities['text'][all_entities['text'].str.contains("58 to 66")]

214985    58 to 66 (even numbers) great portland street,...
214986    58 to 66 (even numbers) great portland street,...
214987    58 to 66 (even numbers) great portland street,...
214988    58 to 66 (even numbers) great portland street,...
214989    58 to 66 (even numbers) great portland street,...
214990    58 to 66 (even numbers) great portland street,...
214991    58 to 66 (even numbers) great portland street,...
214992    58 to 66 (even numbers) great portland street,...
214993    58 to 66 (even numbers) great portland street,...
214994    58 to 66 (even numbers) great portland street,...
Name: text, dtype: object

In [8]:
#pivot the columns so that each label class is it's own column and the value in the column is the text
all_entities['index'] = all_entities.index
df = all_entities[['index', 'label', 'label_text']].pivot(index='index',columns='label',values='label_text')
#add the datapoint_id back in for each of joining
df = pd.concat([all_entities['datapoint_id'], df], axis=1)
df

,datapoint_id,building_name,city,number_filter,postcode,street_name,street_number,unit_id,unit_type
0,0,NaN,NaN,NaN,NaN,nel pan lane,NaN,NaN,NaN
1,0,NaN,leigh,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NaN,NaN,NaN,wn7 5jt,NaN,NaN,NaN,NaN
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,flat
4,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...
357947,94086,NaN,NaN,NaN,ec3a 7ba,NaN,NaN,NaN,NaN
357948,94087,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
357949,94087,NaN,NaN,NaN,NaN,south quay square,NaN,NaN,NaN
357950,94087,NaN,london,NaN,NaN,NaN,NaN,NaN,NaN
